In [1]:
import torch
import torchvision
import numpy as np
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
import sys
from tqdm import tqdm
import numpy as np

sys.path.append("/workspaces/ood/")

%load_ext autoreload
%autoreload 2
device = torch.device("cuda")
torch.cuda.is_available()

True

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

In [3]:
cifar_data_train = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform
)

In [4]:
wide_resnet50 = torchvision.models.wide_resnet50_2(pretrained=False).to(device)
wide_resnet50.load_state_dict(
    torch.load(
        "/workspaces/ood/data/models/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth"
    )
)

for name, param in wide_resnet50.named_parameters():
    param.requires_grad = False

In [6]:
from feature_extractor import FeatureExtractor

layers = ["layer2", "layer3", "layer4"]
encoder = FeatureExtractor(wide_resnet50, layers)

In [7]:
encoder.out_dims

[512, 1024, 2048]

In [5]:
from fastflow.extract_features import create_feature_dataset

model_name = "wide_resnet50"
dataset_name = "cifar10train"
layers = ["layer2", "layer3", "layer4"]
out_dims = [[512, 28, 28], [1024, 14, 14], [2048, 7, 7]]
num_images_per_class = 1000
out_name = (
    "../data/feature_maps/"
    + "_".join([dataset_name] + [model_name] + layers)
    + f"_{num_images_per_class}_img_per_class.npz"
)

In [6]:
create_feature_dataset(
    model=wide_resnet50,
    layers=layers,
    out_dims=out_dims,
    dataset=cifar_data_train,
    num_images_per_class=num_images_per_class,
    out_name=out_name,
    device=device,
)

  0%|          | 0/50000 [00:00<?, ?it/s]

saving to ../data/feature_maps/cifar10train_wide_resnet50_layer2_layer3_layer4_1000_img_per_class.npz...


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
 21%|██▏       | 10716/50000 [01:53<06:54, 94.76it/s] 


In [7]:
from fastflow.extract_features import create_feature_dataset

cifar_data_test = torchvision.datasets.CIFAR10(
    "../data/cifar10", download=False, transform=transform, train=False
)

model_name = "wide_resnet50"
dataset_name = "cifar10test"
layers = ["layer2", "layer3", "layer4"]
out_dims = [[512, 28, 28], [1024, 14, 14], [2048, 7, 7]]
num_images_per_class = 100
out_name = (
    "../data/feature_maps/"
    + "_".join([dataset_name] + [model_name] + layers)
    + f"_{num_images_per_class}_img_per_class.npz"
)

create_feature_dataset(
    model=wide_resnet50,
    layers=layers,
    out_dims=out_dims,
    dataset=cifar_data_test,
    num_images_per_class=num_images_per_class,
    out_name=out_name,
    device=device,
)

 12%|█▏        | 1215/10000 [00:11<01:22, 106.52it/s]


all classes are computed
saving to ../data/feature_maps/cifar10test_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npz...


In [8]:
from fastflow.extract_features import create_feature_dataset

svhn_data_test = torchvision.datasets.SVHN(
    "../data/svhn", download=False, transform=transform, split="test"
)

model_name = "wide_resnet50"
dataset_name = "svhntest"
layers = ["layer2", "layer3", "layer4"]
out_dims = [[512, 28, 28], [1024, 14, 14], [2048, 7, 7]]
num_images_per_class = 100
out_name = (
    "../data/feature_maps/"
    + "_".join([dataset_name] + [model_name] + layers)
    + f"_{num_images_per_class}_img_per_class.npz"
)

create_feature_dataset(
    model=wide_resnet50,
    layers=layers,
    out_dims=out_dims,
    dataset=cifar_data_test,
    num_images_per_class=num_images_per_class,
    out_name=out_name,
    device=device,
)

 12%|█▏        | 1215/10000 [00:12<01:27, 100.91it/s]


all classes are computed
saving to ../data/feature_maps/svhntest_wide_resnet50_layer2_layer3_layer4_100_img_per_class.npz...


## Split files

In [14]:
from fastflow.extract_features import split_layers

model_name = "wide_resnet50"
dataset_name = "cifar10train"
layers = ["layer2", "layer3", "layer4"]
out_dims = [[512, 28, 28], [1024, 14, 14], [2048, 7, 7]]
num_images_per_class = 1000
out_name = (
    "../data/feature_maps/"
    + "_".join([dataset_name] + [model_name] + layers)
    + f"_{num_images_per_class}_img_per_class.npz"
)
split_layers(out_name, layers)

100%|██████████| 3/3 [03:05<00:00, 61.93s/it] 


In [9]:
test = np.load(
    "/workspaces/ood/data/feature_maps/cifar10test_wide_resnet50_layer2_100_img_per_class.npy"
)

In [ ]:
# from feature_extractor import FeatureExtractor
# encoder = FeatureExtractor(wide_resnet50, layers)
# encoder.eval()

# preds = {layers[i] : np.zeros([num_images_per_class * 10]+out_dims[i], dtype=np.float32) for i in range(len(layers))}
# label_counts = {i:0 for i in range(10)}
# label_not_finish = [True for _ in range(10)]
# i = 0
# for image, label in tqdm(cifar_data_train):
#     if i > 140:
#         break
#     if not any(label_not_finish):
#         break
#     if label_counts[label] >= num_images_per_class:
#         label_not_finish[label] = False
#         continue

#     features = encoder(torch.unsqueeze(image.to(device), dim=0))
#     for layer in features.keys():
#         preds[layer][i] = features[layer].detach().cpu().numpy()[0]
#     i+=1
#     label_counts[label]+=1